In [1]:
#IMPORT STATEMENTS
import selenium
import numpy as np
import warnings
warnings.filterwarnings('ignore')  # supress scikit 'future warnings'
import pandas as pd
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from scipy.optimize import linear_sum_assignment
import matplotlib         
from matplotlib import pyplot as plt
from scipy.stats import norm
from scipy.stats import norm, kurtosis
from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn import svm
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from scipy.interpolate import make_interp_spline, BSpline
from scipy import stats
import math
import pickle
import random
import copy
import itertools
from dataclasses import dataclass
random.seed(42)
import torch
import torch.nn as nn
#from basketball_reference_scraper import ask_matches
from basketball_reference_scraper.teams import get_roster, get_team_stats, get_opp_stats, get_roster_stats, get_team_misc
from basketball_reference_scraper.players import get_stats, get_game_logs, get_player_headshot, get_player_suffix, lookup

In [2]:
# Import the courses as pandas
# Import the word list that we are using as a dictionary
df = pd.read_csv("course_df.csv")
words = pd.read_csv("words.csv")
word_list = [word for word in words]

# List of words to remove because they are very common
# and don't add anything to similarity
remove_word_list = ["the", "be", "to", "of", "and", "a", "in", "as", "that", "for", "not", "on", "class", "he", "as", "this", "but", "one", "such", "students", "will", "have", "us", "topic", "topics", "include", "course", "do", "into", "is", "like", "prerequisite", "or", "at", "use", "by"]

In [3]:
# Get the number of classes and number of words in dict
num_classes = len(df.index)
num_words = len(word_list)
df # Print the dataframe for visualization

,Unnamed: 0,Title,Description
0,AA 47SI,Why Go To Space?,Why do we spend billions of dollars exploring ...
1,AA 93,Building Trust in Autonomy,Preparatory course for Bing Overseas Studies s...
2,AA 100,Introduction to Aeronautics and Astronautics,This class introduces the basics of aeronautic...
3,AA 102,Introduction to Applied Aerodynamics,This course explores the fundamentals of the b...
4,AA 103,Air and Space Propulsion,This course is designed to introduce the stude...
...,...,...,...
14794,WELLNESS 183,"Financial Wellness for a Healthy, Long Life","This course will ground you in the knowledge, ..."
14795,WELLNESS 191,Peer Education on Comprehensive Sexual Health,Presented by the Sexual Health Peer Resource C...
14796,WELLNESS 192,Peer Support: Creating Spaces for Healing and ...,Social support is an integral part of the huma...
14797,WELLNESS 198,Directed Reading and Individual Studies - Well...,Translate theoretical knowledge and acquired s...


In [4]:
# USEFUL HELPER FUNCTIONS CELL

# Normalize function -> Takes in np array and returns
# normalized version of this array
def normalize(np_arr):
    return np_arr/np.sqrt(np.sum(np.square(np_arr)))

# Split number function -> Takes in course number and 
# returns tuple of number with any letter after
# Example: 43S -> (43, 'S')
def split_number(course_num):
    index = 1 # Start index at 1

    # Continue to iterate index until end of digits
    while index <= len(course_num) and course_num[:index].isdigit():
        index += 1

    # Return here if no additional characters after number
    return (course_num[:index-1], course_num[index-1:])

    # Continue to loop in case of characters
    index = len(course_num) - 1
    while index >= 0:
        if not(chr(course_num[index]).isdigt()):
            while index >= 0 and not(chr(course_num[index]).isdigit()):
                index -= 1

            return (course_num[:index+1], course_num[index+1:])

        index -= 1

    # Return only course number with no letter
    return (course_num, "")

# Convert an input depending on how it is inputted
def convert_input(input_course):
    # Split based on space if it exists
    pieces_list = input_course.split()
    
    # If there are two pieces from splitting return
    # the two pieces
    if len(pieces_list) == 2:    
        return pieces_list[0], pieces_list[1]
    
    # Use a split index to find where department changes to number
    split_index = -1
    for index in range(len(input_course)):
        if input_course[index].isdigit():
            split_index = index
            break
    
    # Return the the two parts if using a split_index
    return input_course[:split_index], input_course[split_index:]

In [5]:
# Initialize different dictionaries for usage later on
# Dictionaries include:
# Class to description word vector mapping (un)normalized vec
# Class to name/title word vector mapping (un)normalized
# Class to pure name mapping
class_to_vec_dict = {}
class_to_nam_dict = {}
class_to_vec_dict_norm = {}
class_to_nam_dict_norm = {}
class_to_name = {}

# Loop through all the classes that are possible
for class_index in range(num_classes):
    # Get the current class, it's name/title, and description
    curr_class = str(df.iloc[class_index]['Unnamed: 0'])
    name = str(df.iloc[class_index]['Title'])
    desc = str(df.iloc[class_index]['Description'])
    
    # Split up the name and the description and 
    # initialize dicts for this one specific word
    # Contains the word count of each word in the description
    # Contains the word count of each word in the name/title
    desc_word_dict = {}
    name_word_dict = {}
    split_desc = desc.split(" ")
    split_name = name.split(" ")
    
    # If there are less than 20 words in a description just skip
    if len(split_desc) < 20:
        continue
    
    # Loop through all the words in the description
    for word in split_desc:
        # Lowercase the word and add to class's description dict
        new_word = word.lower()
        desc_word_dict[new_word] = desc_word_dict.get(new_word, 0) + 1
    
    # Loop through all the words in the name
    for word in split_name:
        # Lowercase the word and add to class's name dict
        new_word = word.lower()
        name_word_dict[new_word] = name_word_dict.get(new_word, 0) + 1
    
    # Initialize a vector for the description's words and a vector for the name/title's words
    vec = np.array([0 for i in range(num_words)])
    name_vec = np.array([0 for i in range(num_words)])
    
    # Go through every word in the word list and fill that position
    # in the vector IF it is NOT in the removed word list
    for index, word in enumerate(word_list):
        if word in remove_word_list:
            continue
        vec[index] = desc_word_dict.get(word, 0)
        name_vec[index] = name_word_dict.get(word, 0)
    
    # Add found vectors into the respective dictionaries
    class_to_vec_dict[curr_class] = np.array(vec)
    class_to_nam_dict[curr_class] = np.array(name_vec)
    class_to_vec_dict_norm[curr_class] = normalize(np.array(vec))
    class_to_nam_dict_norm[curr_class] = normalize(np.array(name_vec))
    class_to_name[curr_class] = name

    # Print gauges to identify progress made in this lengthy process
    if class_index%1000 == 0:
        print("Completed: " + str(class_index))
    

Completed: 0
Completed: 1000
Completed: 2000
Completed: 3000
Completed: 4000
Completed: 5000
Completed: 6000
Completed: 7000
Completed: 8000
Completed: 9000
Completed: 10000
Completed: 11000
Completed: 12000


In [6]:
# Helper function that prints a class's name AND description vector 
# Prints its normalized version if parameter is True and vice versa
def printer(class_name, normalized = True):
    desc_dict_vec = {}
    name_dict_vec = {}
    
    # Change the description and name vector dictionaries based
    # on wether the normalized parameter is True or False
    if normalized == True:
        desc_dict_vec = class_to_vec_dict_norm
        name_dict_vec = class_to_nam_dict_norm
    else:
        desc_dict_vec = class_to_vec_dict
        name_dict_vec = class_to_nam_dict
    
    # Hold the description word vector and the name word vector
    temp_desc_vec = desc_dict_vec[class_name]
    temp_name_vec = name_dict_vec[class_name]

    # Print out each word in the name along with its corresponding value
    print("NAME INFO:")
    for ind, val in enumerate(temp_name_vec):
        if val == 0:
            continue
        
        print("Word: " + str(word_list[ind]) + " occurs: " + str(val) + " times")
        
    print()
    
    # Print out each word in the description laong with corresponding value
    print("DESC INFO:")
    for ind, val in enumerate(temp_desc_vec):
        if val == 0:
            continue
        
        print("Word: " + str(word_list[ind]) + " occurs: " + str(val) + " times")

In [7]:
# Calculate the similarity score between two classes given parameters
def similarity_score(class_a, class_b, name_mult = 1, desc_mult = 1, normalized = True):
    # Initialize the description and name dictionaries/mappings based on
    # whether using normalized scores or not
    desc_dict_vec = {}
    name_dict_vec = {}
    
    if normalized == True:
        desc_dict_vec = class_to_vec_dict_norm
        name_dict_vec = class_to_nam_dict_norm
    else:
        desc_dict_vec = class_to_vec_dict
        name_dict_vec = class_to_nam_dict
    
    # Get the name and description word vectors for both classes
    class_a_desc_vec = desc_dict_vec.get(class_a, [])
    class_a_name_vec = name_dict_vec.get(class_a, [])
    class_b_desc_vec = desc_dict_vec.get(class_b, [])
    class_b_name_vec = name_dict_vec.get(class_b, [])
    
    # Error checking if either class doesn't exist in the mapping
    if len(class_a_desc_vec) == 0 or len(class_a_name_vec) == 0 or len(class_b_desc_vec) == 0 or len(class_b_name_vec) == 0:
        return "Please input a valid class"
    
    # Calculate the score by using the description distance and name distance
    desc_dist = np.sum(np.square(class_a_desc_vec - class_b_desc_vec))
    name_dist = np.sum(np.square(class_a_name_vec - class_b_name_vec))
    score = desc_mult * desc_dist + name_mult * name_dist
    
    # Return the score
    return score

In [8]:
# This is a single recommender in that it takes in a class (and parameteres)
# and outputs the closest other class to this given class
def single_recommender(class_name, desc_mult = 1, name_mult = 1, normalized = True, OSP_prune = True, lower_prune = True):
    # Initialize and set the class to vector mappings based on whether
    # Normalization is to be used or not
    desc_dict_vec = {}
    name_dict_vec = {}
    
    if normalized == True:
        desc_dict_vec = class_to_vec_dict_norm
        name_dict_vec = class_to_nam_dict_norm
    else:
        desc_dict_vec = class_to_vec_dict
        name_dict_vec = class_to_nam_dict
    
    # Get the description and name word vector for the inputted class
    given_class_vec = desc_dict_vec.get(class_name, [])
    given_class_name_vec = name_dict_vec.get(class_name, [])
    
    # If the class is not found use error type return
    if len(given_class_vec) == 0 or len(given_class_name_vec) == 0:
        return "Please input a valid class"
    
    # Initialize best possible class and its corresponding score for storage
    best_class = "No Class LOL"
    best_score = float('inf')
    
    # Get the inputted class's words in name, department, and class number
    given_class_list = class_name.split(" ")
    given_class_department = given_class_list[0]
    given_class_numerical = split_number(given_class_list[1])
    
    # Loop through all the possible classes in the description word vector dictionary
    for pot_class, pot_vec in desc_dict_vec.items():
        # Get information about the other class (name, department)
        pot_class_list = pot_class.split(" ")
        pot_class_department = pot_class_list[0]
        pot_class_numerical = split_number(pot_class_list[1])
        
        # Prune out classes that are OSP or are lower numbered in the same department
        # It makes no sense to recommend a lower class (usually) since it is assumed
        # to have probably been taken
        if OSP_prune and pot_class_department[:3] == "OSP":
            continue
        if lower_prune and pot_class_department == given_class_department and pot_class_numerical[0] <= given_class_numerical[0]:
            continue
        
        # The name vector of the potential class
        pot_name_vec = name_dict_vec.get(pot_class, [])
        
        # If this class does not exist, error print and return
        if len(pot_name_vec) == 0:
            print("ERROR FAIL")
            return
        
        # If the potential class is the same as the original class
        # Or the description vectors are identical (indicates cross-department course)
        # Skip this current example
        if pot_class == class_name or np.array_equal(pot_vec, given_class_vec):
            continue
        
        # Calculate the score by using the description distance and name distance
        desc_dist = np.sum(np.square(given_class_vec - pot_vec))
        name_dist = np.sum(np.square(given_class_name_vec - pot_name_vec))
        score = desc_mult * desc_dist + name_mult * name_dist
        
        # If the new score is better than the best score
        # Then replace the best_class with this curr class
        # And also replace the best_score with this score
        if score < best_score:
            best_class = pot_class
            best_score = score
     
    # Return the best matching class
    return best_class

In [9]:
# Helper function that is very similar to single_recommender
# Only difference is that this function outputs a list of best
# matches that is equal to the amount parameter
def general_recommender(class_name, amount = 1, desc_mult = 1, name_mult = 1, normalized = True, OSP_prune = True, lower_prune = True):
    # Initialize and set the class to vector mappings based on whether
    # Normalization is to be used or not
    desc_dict_vec = {}
    name_dict_vec = {}
    
    if normalized == True:
        desc_dict_vec = class_to_vec_dict_norm
        name_dict_vec = class_to_nam_dict_norm
    else:
        desc_dict_vec = class_to_vec_dict
        name_dict_vec = class_to_nam_dict
    
    # Get the description and name word vector for the inputted class
    given_class_vec = desc_dict_vec.get(class_name, [])
    given_class_name_vec = name_dict_vec.get(class_name, [])
    
    # If the class is not found use error type return
    if len(given_class_vec) == 0 or len(given_class_name_vec) == 0:
        return "Please input a valid class"
    
    # Initialize a list that holds the top (amount) of best matches
    q = []
    
    # Get the inputted class's words in name, department, and class number
    given_class_list = class_name.split(" ")
    given_class_department = given_class_list[0]
    given_class_numerical = split_number(given_class_list[1])
    
    # Loop through all the possible classes in the description word vector dictionary
    for pot_class, pot_vec in desc_dict_vec.items():
        pot_class_list = pot_class.split(" ")
        pot_class_department = pot_class_list[0]
        pot_class_numerical = split_number(pot_class_list[1])
        
        # Prune out classes that are OSP or are lower numbered in the same department
        # It makes no sense to recommend a lower class (usually) since it is assumed
        # to have probably been taken
        if OSP_prune and pot_class_department[:3] == "OSP":
            continue
        if lower_prune and pot_class_department == given_class_department and pot_class_numerical[0] <= given_class_numerical[0]:
            continue
        
        # The name vector of the potential class
        pot_name_vec = name_dict_vec.get(pot_class, [])
        
        # If this class does not exist, error print and return
        if len(pot_name_vec) == 0:
            print("ERROR FAIL")
            return "Please input a valid class"
        
        # If the potential class is the same as the original class
        # Or the description vectors are identical (indicates cross-department course)
        # Skip this current example
        if pot_class == class_name or np.array_equal(pot_vec, given_class_vec):
            continue
            
        check = False
          
        # Calculate the score by using the description distance and name distance
        desc_dist = np.sum(np.square(given_class_vec - pot_vec))
        name_dist = np.sum(np.square(given_class_name_vec - pot_name_vec))
        score = (desc_mult * desc_dist + name_mult * name_dist)
        
        # If this exact score already exists in the array, skip this example (identical course)
        for el in q:
            if el[1] == score:
                check = True
                break
        if check:
            continue
        
        # Append this class and its score to the best array
        q.append([pot_class, score])
        
        # Re-sort the array
        q.sort(key = lambda x: x[1])
        
        # If there are more elements, pop the largest (worst) score
        if len(q) > amount:
            q.pop()
     
    # Return the array
    return q

In [10]:
###################################################################################
###################################################################################
#### EVERYTHING BELOW THIS POINT IS USED FOR TESTING CLASSES AND PRODUCING DATA ####
###################################################################################
###################################################################################

In [11]:
# Uses the similarity score helper function to
# compute the (non)similarity between two given classes
class_a = "CS 221"
class_b = "CS 224U"
name_mult = 0
desc_mult = 1
normalize = True

# Compute (non)similarity/distance and print out
similarity = similarity_score(class_a, class_b, name_mult = name_mult, desc_mult = desc_mult, normalized = normalize)
print("Class A: " + str(class_a))
print("Class B: " + str(class_b))
print("Non-Similarity (Distance) Score: " + str(similarity))

Class A: CS 221
Class B: CS 224U
Non-Similarity (Distance) Score: 1.1506325664882475


In [12]:
# Utilizes the general recommender helper function 
# the the best (amount) classes in terms of smallest
# similarity scores
# Below defines the parameters that we are going to use
taken_course = "CS 221"
amount = 10
normalize = True
name_mult = 0
desc_mult = 1
OSP_prune = True
lower_prune = True

# Get the list of best courses along with the similarity score
# Print out the results
recommended_course = general_recommender(taken_course, amount = amount, name_mult = name_mult, desc_mult = desc_mult, normalized = normalize, OSP_prune = OSP_prune, lower_prune = lower_prune)
print("GIVEN COURSE: " + str(taken_course))
print("RECOMMENDED COURSES: " + str(recommended_course))

GIVEN COURSE: CS 221
RECOMMENDED COURSES: [['CS 224U', 1.1506325664882475], ['CS 252', 1.1617067563733854], ['CS 276', 1.273297848181444], ['CS 329E', 1.2912438810716917], ['CS 229M', 1.2995620894160358], ['MATH 151', 1.3262784631467848], ['CS 294W', 1.3811472522447241], ['CS 50', 1.4336206241560854], ['CS 398', 1.4342832325333572], ['CS 325B', 1.4532731731981823]]


In [13]:
# This cell utilizes the single recommender function to
# return the best possible match with the given course
# The following below are the parameters to use
taken_course = "CS 221"
printer_on_course = True
normalize = True
name_mult = 0.1
desc_mult = 1
OSP_prune = True
lower_prune = True

# Holds the recommended course and then prints the result
recommended_course = single_recommender(taken_course, name_mult = name_mult, desc_mult = desc_mult, normalized = normalize, OSP_prune = OSP_prune, lower_prune = lower_prune)
print("GIVEN COURSE: " + str(taken_course))
print("RECOMMENDED COURSE: " + str(recommended_course))

# If the printer_on_course is true it prints
# a dense representation of the description and 
# name vectors of both of the classes
if printer_on_course:
    print()
    print()
    printer(taken_course, normalized = normalize)
    print()
    print()
    printer(recommended_course, normalized = normalize)

GIVEN COURSE: CS 221
RECOMMENDED COURSE: CS 224U


NAME INFO:
Word: artificial occurs: 0.5773502691896258 times
Word: principles occurs: 0.5773502691896258 times
Word: techniques occurs: 0.5773502691896258 times

DESC INFO:
Word: ai occurs: 0.10314212462587934 times
Word: artificial occurs: 0.10314212462587934 times
Word: before occurs: 0.10314212462587934 times
Word: building occurs: 0.10314212462587934 times
Word: comfort occurs: 0.10314212462587934 times
Word: computation occurs: 0.10314212462587934 times
Word: concepts occurs: 0.20628424925175867 times
Word: constraint occurs: 0.10314212462587934 times
Word: cs occurs: 0.618852747755276 times
Word: decision occurs: 0.10314212462587934 times
Word: decisions occurs: 0.10314212462587934 times
Word: drive occurs: 0.10314212462587934 times
Word: focuses occurs: 0.10314212462587934 times
Word: game occurs: 0.10314212462587934 times
Word: given occurs: 0.10314212462587934 times
Word: good occurs: 0.10314212462587934 times
Word: graphical 